In [ ]:
# Install necessary libraries
!pip install paho-mqtt folium

In [ ]:
# Import necessary libraries
import pandas as pd
import folium
import paho.mqtt.client as mqtt
import json
from IPython.display import clear_output

In [ ]:
# TTN application and device details
app_id = "<Username from 'Connection credential' of TTN MQTT Integration>" 
access_key = "<Password from 'Connection credential' of TTN MQTT Integration>" 
device_id = "<End device ID from TTN Application>" 

In [ ]:
# Function to handle connection
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("v3/"+ str(app_id) +"/devices/"+ str(device_id) +"/up")

# Function to handle messages
def on_message(client, userdata, message):
    msg = message.payload.decode('utf-8')
    # Parse the JSON data
    data = json.loads(msg)

    # Extract location fields
    latitude = data['uplink_message']['decoded_payload']['latitude']
    longitude = data['uplink_message']['decoded_payload']['longitude']
    # Create a DataFrame to store location
    coord = {'latitude': latitude, 'longitude': longitude}
    df.loc[len(df)] = coord
    plot_locations_on_map()

# Function to plot the locations on a map
def plot_locations_on_map():
  if not df.empty:
          for index, row in df.iterrows():
              folium.Marker([row['latitude'], row['longitude']]).add_to(m)
          clear_output(wait=True)
          display(m)

In [ ]:
# Create an empty DataFrame to store device locations
columns = ['latitude', 'longitude']
df = pd.DataFrame(columns=columns)

# Connect to TTN MQTT
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1, 'prova')
client.username_pw_set(username=app_id, password=access_key)
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker
client.connect("eu1.cloud.thethings.network", 1883, 60)

m = folium.Map(location=[41.110160,16.878887], zoom_start=20)

# Start the MQTT client loop
client.loop_forever()